In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import math
import matplotlib.patches as patches
from arena import *
import scipy.sparse as sps
from scipy.ndimage import gaussian_filter
from sklearn.feature_selection import mutual_info_regression as mi_skl
from scipy import stats
import time
from matplotlib import gridspec
from math import pi
from collections import Counter
import multiprocessing
from place_cells import *
import seaborn as sns
np.seterr(divide='ignore', invalid='ignore')

In [ ]:
Animal_ID = 'H2226' #run for animal #H2226
Ego_Allo = 'P1Allo' #run for Allo Sessions

# If you're using a mac separators are different. Use / instead of \\
codeDir = r'C:\Users\fr87_\OneDrive - University of Edinburgh\arena room files\ANALYSIS 2.0\Code\Gobbo et al. Code' #os.getcwd() 
#len2del = sum([len(i) for i in codeDir.split('/')[-2:]])+1
homeDir = codeDir#[:-len2del]
BEHdir = homeDir + '\\Example Data\\%s\\%s\\BEH' %(Animal_ID, Ego_Allo)
EVEdir = homeDir + '\\Example Data\\%s\\%s\\ALIGNED TRA EVE\\REGISTERED\\Events\\Quality_checked'  %(Animal_ID, Ego_Allo)
TRImet = homeDir + '\\Example Data\\Trial_META_PRE.csv' #this contains the start and end times of PRE (exploration) files. It is located in Example Data

pcDIR = homeDir + '\\Example Data\\Place cell meta' #Where to save place cell table
HDtab = homeDir + '\\Example Data\\Animal Head Direction.csv' #Where to save head direction info
SaveTo = homeDir + '\\Figures\\Place Cells' #need to create this folder if saving is needed

In [ ]:
All_META=pd.DataFrame
coord = 0
for file in os.listdir(EVEdir):
    if file.endswith('.csv') and file[-14:] != 'all_trials.csv':

        E_A = file.split('_')[-4][0]
        Session = file.split('_')[-4][1:]
        Stage = file.split('_')[-3][:3]
        Trial = file.split('_')[-3][-1]

    if file.endswith('events_dlc.csv') and Stage == 'PRE': #<-- focus on Pretraining stages
        print(E_A, Session, Stage, Trial)        
        
        #read in Trial starts and ends
        tr_ts = pd.read_csv(TRImet)
        tr_ts = tr_ts.loc[(tr_ts['Animal'] == Animal_ID) & 
                          (tr_ts['Session'] == int(Session)) & 
                          (tr_ts['Stage'] == 'PRE') & 
                          (tr_ts['Trial'] == int(Trial))]
        tr_st = tr_ts['Trial start'].values[0]
        tr_ed = tr_ts['Trial end'].values[0]      
        
        META = pd.read_csv(EVEdir+'\\'+file)
        META = META.loc[(META['Time (s)'] > tr_st) & (META['Time (s)'] < tr_ed)]

        if coord == 0:
            All_META = META
            #extract arena features (SW & Corners)
            #BehVidname = pd.read_csv(homeDir+'\\Example Data\\Check_status2.csv') # This is the general form for the loop
            #BEHvid = BEHdir+'\\'+ BehVidname.loc[(BehVidname['Animal']==Animal_ID) & (BehVidname['Phase']==Ego_Allo)]['BEH'].values[0]
            BEHvid = BEHdir+'\\H2226_A28_PRE1_BEH.flv' #change here as needed

            bl_x, bl_y, tr_x, tr_y, w_x, w_y, sw_x, sw_y, cxn_x, cxn_y = arena_coodinates(BEHvid)
            max_coord = tr_x + 200
            coord = 1
            
        #Concatenate df
        All_META = pd.concat([All_META, META], ignore_index=True)
All_META = All_META.reset_index(drop=True)

#### Place cell detection

In [ ]:
def find_place_cells(All_META, n_sessions, xdim, ydim, pix_cm, pf_cm, n_events, perc_thresh):
    #Read in animal place cell df
    pcANI = pd.read_csv(os.path.join(pcDIR, Animal_ID + '_place_cell_meta.csv'))
    
    #Only analyse moving points # INCLUDE SPEED FUNCTION HERE
    mov_meta = All_META[All_META['Movement status']=='moving']

    #Isolate cell IDS
    cell_IDS = [col for col in mov_meta if col.startswith(' C')]
        
    #pcs = []; Percs = []; SI =[]
    for i, cell in enumerate(cell_IDS):
        print(i+1,'/',len(cell_IDS))
        if len(pcANI[pcANI['Cell IDS'] == cell]) == 0:
            criteria = np.zeros(4);
            
            #CONDITION 0: N sessions
            cell_active = mov_meta[mov_meta[cell].notnull()]
            if len(np.unique(cell_active['Session'])) >= n_sessions:
                print(cell)
                ev = cell_active[cell].values
                ev[ev > 0] = 1 # OPTIONAL: Binarized events
                ev = ev.T

                #Bin and linearize occupancy
                x = mov_meta['cap_x'].loc[cell_active.index].values.astype(float)
                y = mov_meta['cap_y'].loc[cell_active.index].values.astype(float)
                linS, occMap, nBnx, nBny, bx, by = arena_binned(x = x, y = y,
                                                                xdim = xdim, ydim = ydim,
                                                                pix_cm = pix_cm, pf_cm = pf_cm)
                #Calculate place map
                rateMap = sps.csr_matrix((ev,(bx,by)),shape=(nBnx,nBny),dtype=float).todense()
                placeMap = np.divide(rateMap,occMap); placeMap[np.isnan(placeMap)] = 0

                #CONDITION 1: Exclude cells that have less than n events
                if  len([e for e in ev if e > 0]) > n_events:

                    criteria[0] = int(1)

                    #Calculate SI and compare to distribution
                    res_eve = list(np.array(ev.reshape(-1,1)).astype(float))
                    si = mi_skl(res_eve, linS, discrete_features=True,n_neighbors = 3)
                    si_shuf = SI_shuffles(np.array(res_eve), linS, iterations = shuffles)

                    #Calculate percentile
                    perc = stats.percentileofscore(np.array(si_shuf).T[0], si)
                    #Percs.append(perc)

                    #Account for n traversals random bursting cells
                    s_placeMap = gaussian_filter(placeMap, sigma = 1)
                    sn_placeMap = np.array(s_placeMap)/np.max(s_placeMap)
                    sn_placeMap[sn_placeMap < 0.2] = 0
                    pf_idxs = np.argwhere(sn_placeMap > 0)
                    S_pf = np.vstack((pf_idxs[:,0],pf_idxs[:,1]))
                    linSpf = np.ravel_multi_index(S_pf,(nBnx,nBny))
                    tr, rdm_burst = bursting_check(linSpf, linS, ev, tr_delay)

                    # CONDITION 2: cell SI > Threshold are considered a Place cell
                    if perc >= perc_thresh:
                        criteria[1] = int(1)

                    #CONDITION 3: Number of traversals required   
                    if tr > n_tr: 
                        criteria[2] = int(1)

                    #CONDITION 4: number of traversals with cell firing  
                    if rdm_burst >= tr_thresh: 
                        criteria[3] = int(1)

                    #If all place cell criteria, then place cell identified
                    if sum(criteria) == 4:
                        print(cell_IDS[i], 'Is a place cell with %d traversals' %tr)
                        pcs = 1

                    else:
                        pcs = 0
                else:
                    pcs = 0
                    si = 0
                    perc = 0
            else:
                pcs = 0
                si = 0
                perc = 0
            
            print(cell, str(criteria), perc, pcs, si)
            #Add stage to DF
            PC_st = pd.DataFrame({'Cell IDS':[cell],
                                  'Criteria':[str(criteria)],
                                  'Percentile':[perc],
                                  'Place cells':[pcs],
                                  'SI':[si]})
            
            pcANI = pd.concat([pcANI, PC_st], ignore_index=True)
            pcANI.to_csv(os.path.join(pcDIR, '%s_place_cell_meta.csv' %(Animal_ID)), index=False)
    
    return pcANI
                
if __name__ == '__main__':
    #PLACE CELL INCLUSION CRITERIA
    n_sessions = 3 # CONDITION 0: Cell has to fire in at least n sessions 
    n_events = 3 # CONDITION 1: set n events for a cell to be considered
    perc_thresh = 99 # CONDITION 2: Set place cell spatial info percentile 
    n_tr = 3 # CONDITION 3: set number of traversals to be considered
    tr_thresh = 20 # CONDITION 4: % traversals that cell fires <-- accounts for rdm bursting

    #Run place cell code
    xdim = [900, -100] # Set arena x dimensions (northwest - (-20,-20), 320x320)
    ydim = [900,-100] # Set Arena y dimensions
    pix_cm = 5.2 # Set how many pixels = 1 cm
    pf_cm = 10 # Set bin size for place field (cms)
    shuffles = 10 # set number of shuffles to create SI distribution
    tr_delay = 40 # acceptable delay between traversals (frames)

    #NOTE: one session is provided as example so need to run n_sessions=1
    n_sessions=1
    
    PC_st = find_place_cells(All_META,n_sessions, xdim, ydim, pix_cm, pf_cm,n_events, perc_thresh)

In [ ]:
pcANI = pd.read_csv(os.path.join(pcDIR, Animal_ID + '_place_cell_meta.csv'))

pcANI[pcANI['Cell IDS'] == cell]

#### Event maps

In [ ]:
#Only analyse moving points # INCLUDE SPEED FUNCTION HERE
mov_meta = All_META[All_META['Movement status']=='moving']

#Isolate cell IDS
cell_IDS = [col for col in mov_meta if col.startswith(' C')]

for i, cell in enumerate(cell_IDS):
    print(i+1,'/',len(cell_IDS))
    
    #CONDITION 0: N sessions
    cell_active = mov_meta[mov_meta[cell].notnull()]
    if len(np.unique(cell_active['Session'])) >= n_sessions:
        print(cell)

        #Bin and linearize occupancy
        x = mov_meta['cap_x'].loc[cell_active.index].values.astype(float)
        y = mov_meta['cap_y'].loc[cell_active.index].values.astype(float)
        
        #Extract event locations
        ev_idx = cell_active[cell_active[cell]>0]
        ev_x = ev_idx['cap_x'].values; ev_y = ev_idx['cap_y'].values
        
        #Extract cell events
        ev = cell_active[cell].values
        ev[ev > 0] = 1 # OPTIONAL: Binarized events
        ev = ev.T
        
        #Calculate occupancy map
        linS, occMap, nBnx, nBny, bx, by = arena_binned(x = x, y = y,
                                                        xdim = xdim, ydim = ydim,
                                                        pix_cm = pix_cm, pf_cm = pf_cm)
        #Calculate place map
        rateMap = sps.csr_matrix((ev,(bx,by)),shape=(nBnx,nBny),dtype=float).todense()
        placeMap = np.divide(rateMap,occMap); placeMap[np.isnan(placeMap)] = 0
        pc_plot1 = gaussian_filter(np.array(placeMap), sigma = 1)
        max_pc_plot = np.max(pc_plot1)
        pc_plot = pc_plot1/max_pc_plot
        pc_plot[pc_plot <= 0.05] = 0
        occ_nan = np.argwhere(occMap == 0)

        fig = plt.figure(figsize=(6,3))
        fig.suptitle(cell + ' ' + str(np.unique(cell_active['Session'])), fontsize=10)
        fig.tight_layout()             
        gs = gridspec.GridSpec(1, 2, width_ratios=[3, 3])
        plt.subplot(gs[0])
        pc_plot[np.ix_(occ_nan[:,0]),occ_nan[:,1]] = np.nan
        plt.imshow(pc_plot.T, aspect="auto", cmap='viridis')
        plt.gca().invert_xaxis()
        sns.despine()
        plt.axis('off')

        ax = plt.subplot(gs[1])
        plt.plot(x,y,c='lightgray', linewidth=0.1, zorder=0)
        plt.scatter(ev_x, ev_y, c='r', s=2)
        plt.scatter(sw_x, sw_y, c='lightblue', s=50, alpha = 0.2, edgecolors='k') #plot sws
        rect = patches.Rectangle((bl_x, bl_y), w_x, w_y, linewidth=0.2, edgecolor='k', facecolor='none')
        ax.add_patch(rect) #plot arena
#         ax.invert_yaxis()
        plt.axis('off')
       # plt.savefig(SaveTo + 'Place Maps/%s/%s_PlaceMaps.png' %(Animal_ID, cell[1:]), dpi=200, bbox_inches="tight")
        plt.show()